# User-based Collaborative filtering

This notebook demonstrates a basic Collaborative filtering technuques implemented with MovieLens Datasets.

* Predict unknown rating with a sparse user-item matrix
* Measure similarity with 'Cosine similarity' and 'Pearson correlation coefficient'

In [2]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cosine

## Loading and preprocess the datasets

We use 100K MovieLens datasets.


In [3]:
path = './datasets/'
ratings = np.array(pd.read_csv(path + "ratings.csv"))#.astype(int)

u_count = int(max(ratings[:,0])+1) # number of users
m_count = int(max(ratings[:,1])+1) # number of movies


# Create user-item rating matrix 
r_mat = np.zeros([u_count,m_count])
        
for i in range(ratings.shape[0]):
    r_mat[int(ratings[i][0]),int(ratings[i][1])] = ratings[i][2]

# Mean of each user's rating
r_mean = np.zeros([u_count])

for i in range(u_count):
    r_mean[i] = np.mean(r_mat[i,np.nonzero(r_mat[i])]) 

/Users/kwoncheol/deep/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/kwoncheol/deep/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


## Normalize technique 1

Extract mean of each user from each user's rating values.

It prevents handling un-rated item as a negative value.

In [4]:
for i in range(u_count):
    r_mat[i,r_mat[i] != 0] = r_mat[i,r_mat[i] != 0] - r_mean[i]

## Normalize technuque 2

Set values bigger than 3 as 1
    values smaller than 3 or not rated as 0.
In this notebook we are not going to use this technique.

In [5]:
#for i in range(self.ratings.shape[0]):
#    self.r_mat[self.ratings[i][0],self.ratings[i][1]] = 0 if self.ratings[i][2]>=3 else 1

## Define Pearson Correlation measure function

There is a PCC function in Scipy but we will define our own PCC function.
It receives indices of two users and returns their PCC value.

In [6]:
def my_pears(a, b): 
    '''
    Pearson Correlation Coefficient function
        
    Arguments:
        a - Index of user A
        b - Index of user B

    '''
    # Transpose the rating matrix for easy to find the items that users(a,b) have rated.
    
    aidx = ratings.T[0] == a
    bidx = ratings.T[0] == b

    br_idx = np.intersect1d(ratings[aidx][:,1],ratings[bidx][:,1]).astype(int)
        
    # Return 0 if no intersect
        
    if(len(br_idx) == 0):
        return 0

    sim = np.sum((r_mat[a,br_idx]-r_mean[a])*(r_mat[b,br_idx]-r_mean[b])) \
        / (np.sqrt(np.sum(np.square(r_mat[a,br_idx]-r_mean[a]))) \
        * np.sqrt(np.sum(np.square(r_mat[b,br_idx]-r_mean[b]))))

    return sim

## Predict users' unknown ratings

Define function that predicts rating of an user and a movie.
1. Measure similarities of users who have rated movie what we want to predict. Then select K most similar users to our user.
2. Predict our user's rating based on K most similar users' ratings.

### There are two predicting methods
1) Divide Sum of ratings of K most similar user by K

2) Divide Sum of product of each user's similiarties and ratings by sum of similarities


In [7]:
def predict_score(u, mov, k=10, sim_met='pearson'):
    '''
    Prediction for item 'mov' of user 'u'
        
    Arguments:
        u - user
        mov - Movie's index     
        k - Number of similar users for prediction (Default = 10)         
        sim_met - similarity measure method (Default = 'pearson')
  
    '''
    
    # Find users who have rated 'mov'
    rated_u = np.array(np.nonzero(r_mat[:,mov]))
        
    if rated_u.shape[1] == 0 or (rated_u.shape[1] == 1 and rated_u[0,0] == u):
        #print("No users who have rated this movie")
        return (0,0)
        
    rated_u = rated_u.flatten()
        
    sims = {}
    for i in range(rated_u.shape[0]):
        if rated_u[i] == u:
            continue
            
        if sim_met is 'pearson':
            sim = my_pears(u,rated_u[i])
        else:
            sim = cosine(r_mat[u,:],r_mat[rated_u[i],:])

        if len(sims) < k:
            sims[rated_u[i]] = sim

        # If the similarity of a current user is smaller than the minimum similarity of 'Sims',
        # then replace minimum user with current user.
        elif sim > sims[min_val]:
            sims.pop(min_val)
            sims[rated_u[i]] = sim

        min_val = min(sims, key=lambda k:sims[k])


    p_ver_1 = np.sum(r_mat[[*sims],mov]) / (rated_u.shape[0] if rated_u.shape[0] < k else k)
    p_ver_2 = np.sum(r_mat[[*sims],mov] * list(sims.values())) / np.sum(list(sims.values()))

    return p_ver_1, p_ver_2

## Example

Let's predict known ratings of 'user 3'. 

In [8]:
# Figure items user 3 have rated out
rated_list = np.array(np.nonzero(r_mat[3,:])).flatten()
rated_list

array([   60,   110,   247,   267,   296,   318,   355,   356,   377,
         527,   588,   592,   593,   595,   736,   778,   866,  1197,
        1210,  1235,  1271,  1378,  1580,  1721,  1884,  2028,  2318,
        2513,  2694,  2702,  2716,  2762,  2841,  2858,  2959,  3243,
        3510,  3949,  5349,  5669,  6377,  7153,  7361,  8622,  8636,
       27369, 44191, 48783, 50068, 58559, 84236])

#### Predict rating of Movie 60 with a 'pearson' methods

In [9]:
#pearson
print('Predicting values : \t',predict_score(3,60))
print('Real rating value :\t', r_mat[3,60])

Predicting values : 	 (-0.3416791353532814, -0.3458529191646994)
Real rating value :	 -0.5686274509803924


#### Predict rating of Movie 60 with a 'cosine' methods

In [10]:
#cosine
print('Predicting values : \t',predict_score(3,60,sim_met='cosine'))
print('Real rating value :\t', r_mat[3,60])

Predicting values : 	 (-1.3883074678646374, -1.383321162066603)
Real rating value :	 -0.5686274509803924


#### Pearson method returns a bit more close result.  Let's do on movie 247.

In [11]:
# pearson
print('Predicting values : \t',predict_score(3,247))
print('Real rating value :\t', r_mat[3,247])

Predicting values : 	 (0.26461344528587805, 0.26247249912838305)
Real rating value :	 -0.06862745098039236


In [12]:
#cosine
print('Predicting values : \t',predict_score(3,247,sim_met='cosine'))
print('Real rating value :\t', r_mat[3,247])

Predicting values : 	 (0.20257315610389953, 0.2033382372547101)
Real rating value :	 -0.06862745098039236


#### Pearson is better on movie 247 as well. Now we are going to measure its error more precisely.

In [13]:
pearson_ver1_errors = 0
pearson_ver2_errors = 0
cosine_ver1_errors = 0
cosine_ver2_errors = 0

for i in range(u_count):
    rated_mov = np.array(np.nonzero(r_mat[i,:])).flatten()
    
    for j in rated_mov:
        pred_val = predict_score(i,j,sim_met='pearson')
        
        pearson_ver1_errors = pearson_ver1_errors + np.sqrt(np.square(pred_val[0] - r_mat[i,j]))
        pearson_ver2_errors = pearson_ver2_errors + np.sqrt(np.square(pred_val[1] - r_mat[i,j]))
        
        pred_val = predict_score(i,j,sim_met='cosine')
        
        cosine_ver1_errors = cosine_ver1_errors + np.sqrt(np.square(pred_val[0] - r_mat[i,j]))
        cosine_ver2_errors = cosine_ver2_errors + np.sqrt(np.square(pred_val[1] - r_mat[i,j]))

In [14]:
print('pearson: \n\tver 1: %d\n\tver 2: %d'%(pearson_ver1_errors,pearson_ver2_errors))
print('cosine: \n\tver 1: %d\n\tver 2: %d'%(cosine_ver1_errors, cosine_ver2_errors))

pearson: 
	ver 1: 63984
	ver 2: 64757
cosine: 
	ver 1: 86580
	ver 2: 87551


#### Pearson method shows a far better results.  
#### And It seems there are not that significant difference between two predict methods we used .